

#**Az elveszett kincs nyomában**


Adottak a szigetek koordinátái, az egyes szigeteken sejtett kincs értéke, valamint a szigeteken kolóniát kialakító kalóz-nemzetek (factions) azonosítója. A szigetek egy részén nincs kolónia, ezek lakatlanok. A lakatlan szigeteket az a kalóz-nemzet uralja, amelyiknek az adott szigethez legközelebb esik kolóniája (a távolságot L1 norma szerint mérik; azonos távolságú legközelebbi kolóniák esetén a kisebb azonosítójú kalóz-nemzet uralja a szigetet). A feladatod, hogy felmérd, hány kalóz-nemzet van jelen a szigetvilágban, melyik uralja a legtöbb szigetet, illetve, hogy melyik szigetet melyik nemzet uralja. Továbbá, meg kell adnod, hogy összesen mennyi kincset sejtenek az egy-egy kalóz-nemzet által uralt szigeteken, mely nemzetek keveredhetnek könnyen viszályba (két közvetlenül szomszédos sziget eltérő nemzetek által uralva), végül, hogy milyen koordináták felelnek meg az elsüllyedt kincses hajó helyzetét leíró nyomoknak. A kincses hajó a legenda szerint olyan tengeri mezőn süllyedt el, amit legalább három oldalról (átlós irányú szomszéd nem számít) szigetek vesznek körbe.

Készítsd el az **IslandsMap** osztályt! Konstruktorának paraméterei a következők:
* **island_coords**: _ndarray(n_islands, 2:[y,x]) of int32_; az egyes szigetek koordinátáit tárolja y,x sorrendben.
* **island_treasure**: _ndarray(n_islands,) of float32_; az egyes szigeteken elrejtett kincsek értékét tárolja.
* **faction_ids**: _ndarray(n_islands,) of int32_; Tárolja az egyes szigetekhez, hogy azok lakatlanok (-1 értékek), vagy valamelyik kalóz-nemzet kolóniája található rajtuk (a kolóniát birtokló kalóz-nemzet azonosítója). A kalóz-nemzetek azonosítói a [0, n_factions-1] intervallumból kerülnek ki.

Implementáld az alábbi metódusokat az **IslandMap** osztályhoz:

* **get_n_factions()**: Visszaadja a különböző kalóznemzetek számát, visszatérési típusa _int_.

* **get_faction_with_most_colonies()**: Megadja annak a kalóznemzetnek az azonosítóját, amelyik a legtöbb kolonizált szigettel rendelkezik, visszatérési típusa _int_. Holtverseny esetén elég az egyik nemzet azonosítóját visszaadni.

* **get_controlling_faction_ids()**: Minden szigethez megadja az azt irányító kalóz-nemzet azonosítóját. Az egyes lakatlan szigeteket az a kalóz-nemzet irányítja, amelyik a szigethez legközelebb eső kolonizált szigettel rendelkezik. Ha a szigethez több, azonos távolságban levő, különböző kalóz-nemzethez tartozó kolónia esik legközelebb, a kisebb azonosítóval rendelkező nemzet irányítja. A távolságot a kalózok az L1-norma szerint mérik. A függvény visszatérési értéke tehát egy, a konstruktorban kapott **faction_ids** tömbbel egyező, _(n_islands,)_ alakú és egyező típusú tömb, azonban a lakatlan szigetekhez nem -1 értékek, hanem a szigeteket irányító kalóznemzetek azonosítója van rendelve.

* **get_faction_treasury()**: Minden kalóz-nemzethez megadja az általuk irányított szigeteken található kincsek értékének összegét. A függvény visszatérési értéke tehát egy _(n_factions,)_ alakú lebegőpontos típusú tömb, melynek az i. eleme az #i azonosítójú kalóz-nemzet által irányított összes szigeten (saját kolóniák és irányított lakatlan szigetek) található kincsek értékének összege.

* **get_factions_with_rival_neighbors()**: Megadja azoknak a kalóz-nemzeteknek az azonosítóját, melyeknek van olyan szigetük (kolonizált, vagy lakatlan), ami egy másik nemzet által uralt szigettel közvetlenül szomszédos (azaz, a távolságuk 1). A függvény visszatérési értéke tehát egy _(n_factions_with_rival_neighbors,)_ alakú, a feltételnek megfelelő kalóz-nemzetek azonosítóit tartalmazó tömb.

* **get_big_treasure_coords()**: Megadja azokat a koordinátákat, ahol a "nagy kincs" a szóbeszéd alapján lehet, _(n_possible_locations, 2:[y,x])_ alakú tömbben. A tömb nem tartalmazhatja ugyanazokat a koordinátákat többször. A "nagy kincset" rejtő koordináták ismertetőjegyei tehát a következők: (1) az adott koordinátán nincs sziget, (2) legalább három, oldalával szomszédos mezőn van sziget.



In [ ]:
from numpy.ma import count
import numpy as np

class IslandsMap:

  def __init__(self, island_coords, island_treasure, faction_ids):
    self.island_coords = island_coords # island_coords: ndarray(n_islands, 2:[yx]) of int32; coordinates of each island on the map
    self.island_treasure = island_treasure # island_treasure: ndarray(n_islands,) of float32; the estimated treasure on each island
    self.faction_ids = faction_ids # faction_ids: ndarray(n_islands,) of int32; the ID of the faction owning the colony on each island; if -1, the island is uninhabited

  def get_n_factions(self):
    return np.unique(self.faction_ids[self.faction_ids!=-1]).shape[0] # -> int

  def get_faction_with_most_colonies(self):
    return np.argmax(np.bincount(self.faction_ids[self.faction_ids!=-1])) # -> int; faction ID which has the most colonies (one of them is enough if multiple factions have equal number of colonies)


  def get_controlling_faction_ids(self):
    distances = np.abs(self.island_coords[:, np.newaxis, :] - self.island_coords)
    distances = distances[self.faction_ids==-1]
    distances = distances.sum(axis=2)

    mask_self = distances==0
    mask_uninhabited = np.tile(self.faction_ids==-1,(distances.shape[0],1))
    mask=mask_self | mask_uninhabited

    masked_distances = np.ma.masked_array(distances, mask=mask)
    closest = np.argmin(masked_distances,axis=1)

    controlling_faction_ids=self.faction_ids.copy()
    controlling_faction_ids[controlling_faction_ids==-1]=self.faction_ids[closest]

    return controlling_faction_ids
    # -> ndarray(n_islands,) of int32; the controlling faction IDs for each island
    #     each island is controlled by the faction with the closest (L1 distance) colony

  def get_faction_treasury(self):
    fc_num=self.get_n_factions()
    fc_masks = self.get_controlling_faction_ids()[:, np.newaxis] == np.arange(fc_num)
    action_treasury = np.sum(self.island_treasure[:, np.newaxis] * fc_masks, axis=0,dtype=int)
    return action_treasury
    # -> ndarray(n_factions,) of float32; the sum of treasures in the islands (incl. colonies) controlled by each faction


  def get_factions_with_rival_neighbors(self):
    controlling_fc=self.get_controlling_faction_ids()
    neighbor_matrix = np.abs(self.island_coords[:, np.newaxis] - self.island_coords).sum(axis=2)==1
    rival_neighbor_mask = (neighbor_matrix & (controlling_fc[:, np.newaxis] != controlling_fc)).any(axis=1)
    factions_with_rival_neighbors = np.unique(controlling_fc[rival_neighbor_mask])
    return factions_with_rival_neighbors
    # -> ndarray(n_factions_with_rival_neighbors,) of int32; the faction IDs who own an island adjacent to an island controlled by another faction


  def get_big_treasure_coords(self):
    possible_locations = []
    neighbor_offsets = np.array([[-1, 0], [1, 0], [0, -1], [0, 1]])

    neighbors = self.island_coords[:, np.newaxis, :] + neighbor_offsets
    neighbor_is_island = np.any(np.all(neighbors == self.island_coords[:, np.newaxis, np.newaxis], axis=-1),axis=0)

    neighbors_of_neighbors=neighbors[~neighbor_is_island]
    neighbors_of_neighbors_offset = neighbors_of_neighbors[:, np.newaxis, :] + neighbor_offsets
    neighbors_of_neighbor_valid= np.any(np.all(neighbors_of_neighbors_offset == self.island_coords[:, np.newaxis, np.newaxis], axis=-1),axis=0)

    counted=np.sum(neighbors_of_neighbor_valid,axis=1)>=3
    possible_locations = np.unique(neighbors_of_neighbors[counted],axis=1)
    return possible_locations
    # -> (n_possible_locations, 2:[y,x])





##Tests

In [ ]:

# Test1

# 0~~X~X1~~~
# ~~X~X2~~~~
# 1~~X~X~~~4
# ~~~X~X2~~~
# ~~~3~~~~~~

# map legend:
#   X - uninhabited island
#   <number> - colonized island with the faction ID
#   ~ - sea tile

_test1_coords = np.array([[0,0],[0,3],[0,5],[0,6],\
                          [1,2], [1,4], [1,5],\
                          [2,0],[2,3],[2,5],[2,9],\
                          [3,3],[3,5],[3,6],\
                          [4,3]], dtype=np.int32)
_test1_treasure = np.arange(15, dtype=np.float32)
_test1_factions = np.array([0,-1,-1,1,-1,-1,2,1,-1,-1,4,-1,-1,2,3], dtype=np.int32)

_test1_map = IslandsMap(_test1_coords, _test1_treasure, _test1_factions)
assert _test1_map.get_n_factions() == 5
assert _test1_map.get_faction_with_most_colonies() in [1, 2]
assert np.all(_test1_map.get_controlling_faction_ids() == [0,0,1,1, 0,2,2, 1,3,2,4, 3,2,2, 3])
assert np.allclose(_test1_map.get_faction_treasury(), [5, 12, 45, 33, 10])
assert np.all(np.sort(_test1_map.get_factions_with_rival_neighbors()) == [1,2])
assert np.all(np.unique(_test1_map.get_big_treasure_coords(), axis=0) == np.unique([[1,3],[0,4],[2,4]], axis=0))  # vector unique sorts vectors


print('TESTS OK')

TESTS OK


##Runtime test

In [ ]:
# Measuring running time

import time

def sample_benchmark():
  # sample code timing
  time0 = time.time()
  for i in range(10):
    _sample_arr = np.random.rand(1000,1000)
    _sample_arr = np.matmul(_sample_arr, _sample_arr.T)
    _sample_idxs = np.random.randint(_sample_arr.size, size=(5000000,))
    _sample_arr = _sample_arr.reshape(-1)[_sample_idxs] ** 2
  time1 = time.time()
  return time1-time0

def time_solution(map_size, island_ratio, colony_ratio, n_factions):
  # Parameters:
  #   map_size: int; length/width of the square map
  #   island_ratio: float; ratio of island tiles
  #   colony_ratio: float; ratio of colonies
  #   n_factions: int; number of factions
  assert 2 < map_size <= 10000
  assert 0. < island_ratio <= 0.5
  assert 0. < colony_ratio <= 0.5
  n_islands = int(map_size*map_size*island_ratio)
  n_colonies = int(map_size*map_size*colony_ratio)
  assert n_islands >= 1
  assert n_colonies <= n_islands
  assert 1 < n_factions <= n_colonies

  print("--- Timing test --- map size: {0}x{0}, n_islands: {1}, n_colonies: {2}, n_factions: {3}".format(map_size, n_islands, n_colonies, n_factions))
  sample_benchmark_time = sample_benchmark()
  print("        Sample benchmark time: ", sample_benchmark_time)

  _island_coords = np.random.choice(map_size*map_size, size=(n_islands,), replace=False).astype(np.int32)
  _island_coords =  np.array(np.unravel_index(_island_coords, (map_size,map_size))).T
  _island_treasure = np.fabs(np.random.normal(0., 100., size=(n_islands,))).astype(np.float32)
  colony_idxs = np.random.choice(n_islands, size=(n_colonies,), replace=False)
  colony_factions = np.random.randint(n_factions, size=(n_colonies,))
  _faction_ids = np.full((n_islands,), fill_value=-1, dtype=np.int32)
  _faction_ids[colony_idxs] = colony_factions
  #
  time0 = time.time()
  map_obj = IslandsMap(_island_coords, _island_treasure, _faction_ids)
  time1 = time.time()
  map_obj.get_faction_with_most_colonies()
  time2 = time.time()
  map_obj.get_controlling_faction_ids()
  time3 = time.time()
  map_obj.get_faction_treasury()
  time4 = time.time()
  map_obj.get_factions_with_rival_neighbors()
  time5 = time.time()
  map_obj.get_big_treasure_coords()
  time6 = time.time()

  print("        Timing test successful!")
  print("            constructor:            ", time1-time0)
  print("            most colonies:          ", time2-time1)
  print("            controlling faction:    ", time3-time2)
  print("            faction treasury:       ", time4-time3)
  print("            rival neighbors:        ", time5-time4)
  print("            big treasure:           ", time6-time5)

  print("        Total time: ", time6-time0)
  print("        Total time (normalized with benchmark): ", (time6-time0)/sample_benchmark_time)

time_solution(map_size=500, island_ratio=.05, colony_ratio=.01, n_factions=50)
time_solution(map_size=200, island_ratio=.3, colony_ratio=.2, n_factions=100)
time_solution(map_size=200, island_ratio=.3, colony_ratio=.01, n_factions=10)



--- Timing test --- map size: 500x500, n_islands: 12500, n_colonies: 2500, n_factions: 50
        Sample benchmark time:  2.6058385372161865
        Timing test successful!
            constructor:             6.9141387939453125e-06
            most colonies:           0.00012254714965820312
            controlling faction:     6.504065990447998
            faction treasury:        6.507306337356567
            rival neighbors:         8.649314641952515
            big treasure:            7.153067588806152
        Total time:  28.813884019851685
        Total time (normalized with benchmark):  11.057432610783904
--- Timing test --- map size: 200x200, n_islands: 12000, n_colonies: 8000, n_factions: 100
        Sample benchmark time:  2.1321046352386475
        Timing test successful!
            constructor:             5.9604644775390625e-06
            most colonies:           0.00018525123596191406
            controlling faction:     3.7668983936309814
            faction treasury: